In [3]:
import requests
from bs4 import BeautifulSoup
from lxml import html
from lxml import etree
import pandas as pd

import sqlite3
import datetime
today_date = datetime.datetime.today().strftime('%Y-%m-%d')

from fantasy_machine import config

SyntaxError: invalid syntax (update_data.py, line 109)

In [ ]:
lineup__odds-item

# Scraping
___
## Get boxes

In [29]:
class daily_lineups(object):
    def __init__(self):
        self.lineup_url = 'https://www.rotowire.com/baseball/daily_lineups.htm'
        self.page = requests.get(self.lineup_url)
        self.tree = html.fromstring(self.page.content)
        self.div_selection = "contains(@class,'lineup is-mlb') and not(contains(@class,'is-tools'))"
        self.info_locations = {'team':"//div[{}][{}]/div[2]/div/div/div[{}]/div[1]/text()",
                               'pitcher':"//div[{}][{}]/div[2]/div[2]/ul[{}]/li[1]/div/a",
                               'players':"//div[{}][{}]/div[2]/div[2]/ul[{}]/li/a",
                               'positions':"//div[{}][{}]/div[2]/div[2]/ul[{}]/li/div[@class='lineup__pos']",
                               'lineup_status':"//div[{}][{}]/div[2]/div[2]/ul[{}]/li[2]",
                               'lineup_odds':"//div[{}][{}]/div[2]/div[3]/div[2]/div[@class='lineup__odds']/div[1]/text()"}
    def main(self):
        games = []
        games_today = len(self.tree.xpath("//div[{}]".format(self.div_selection)))
        teams = [1,2]
        for game_num in range(games_today):
            game_num = game_num + 1
            game = []
            for team in teams:
                team_temp = self.tree.xpath(self.info_locations['team'].format(self.div_selection,game_num,team))
                pitcher_temp = self.tree.xpath(self.info_locations['pitcher'].format(self.div_selection,game_num,team))
                position_temp = self.tree.xpath(self.info_locations['positions'].format(self.div_selection,game_num,team))
                player_temp = self.tree.xpath(self.info_locations['players'].format(self.div_selection,game_num,team))
                lineup_status_temp = self.tree.xpath(self.info_locations['lineup_status'].format(self.div_selection,game_num,team))
                lineup_odds_temp = self.tree.xpath(self.info_locations['lineup_odds'].format(self.div_selection,game_num))
                game += self.get_players(player_temp, position_temp, team_temp, pitcher_temp, lineup_status_temp, game_num,lineup_odds_temp)
            games += game
        return games


    def get_team(self, team_scrape):
        try:
            team = team_scrape[0]
        except:
            team = None
        return team
    def get_pitcher(self, pitcher_scrape):
        pitcher = pitcher_scrape[0].text
        pitcher_id = int(pitcher_scrape[0].attrib['href'].split('?id=')[-1])
        return pitcher, pitcher_id
    def get_lineup_status(self, lineup_status_scrape):
        lineup_status_scrape_class = lineup_status_scrape[0].attrib['class']
        try:
            if 'confirmed' in lineup_status_scrape_class:
                lineup_status = 'Confirmed'
            elif 'expected' in lineup_status_scrape_class:
                lineup_status = 'Expected'
            else:
                lineup_status = 'None'
        except:
            print('status retrieval failure')
            lineup_status = 'None'
        return lineup_status

    def get_players(self, players_scrape, position_scrape, team_scrape, pitcher_scrape, lineup_status_scrape, game_num,lineup_odds_temp):
        team = self.get_team(team_scrape)
        player_list = []
        lineup_odds = ' '.join(lineup_odds_temp[0].split(' ')[1:])
        lineup_status = self.get_lineup_status(lineup_status_scrape)
        pitcher, pitcher_id = self.get_pitcher(pitcher_scrape)
        player_list.append({'fixture':game_num,
                                'player_name':pitcher,
                                'player_id':pitcher_id,
                                'position':'P',
                                'team':team,
                                'lineup_status':lineup_status,
                                'lineup_odds':lineup_odds})
        for player_, position_ in zip(players_scrape, position_scrape):
            position = position_.text
            player_name = player_.attrib['title']
            player_id = int(player_.attrib['href'].split('?id=')[-1])
            player_list.append({'fixture':game_num,
                                'player_name':player_name,
                                'player_id':player_id,
                                'position':position,
                                'team':team,
                                'lineup_status':lineup_status,
                                'lineup_odds':lineup_odds})
        return player_list


In [30]:
gl = daily_lineups()

In [31]:
lineups = pd.DataFrame.from_dict(gl.main()).drop_duplicates()

In [32]:
lineups

,fixture,lineup_odds,lineup_status,player_id,player_name,position,team
0,1,CIN -135,Confirmed,14194,Freddy Peralta,P,MIL
1,1,CIN -135,Confirmed,11764,Ben Gamel,CF,MIL
2,1,CIN -135,Confirmed,11522,Christian Yelich,RF,MIL
3,1,CIN -135,Confirmed,8624,Ryan Braun,LF,MIL
4,1,CIN -135,Confirmed,12734,Travis Shaw,3B,MIL
5,1,CIN -135,Confirmed,12327,Jesus Aguilar,1B,MIL
6,1,CIN -135,Confirmed,9756,Mike Moustakas,2B,MIL
7,1,CIN -135,Confirmed,11386,Manny Pina,C,MIL
8,1,CIN -135,Confirmed,12864,Orlando Arcia,SS,MIL
10,1,CIN -135,Confirmed,14495,Luis Castillo,P,CIN


In [21]:
lineups['game_date'] = today_date
lineups['pull_time'] = pull_time

In [6]:
con = sqlite3.connect(config.lineups_db_path)

In [7]:
lineups.to_sql('daily_lineups', con, if_exists='append', index=False)

In [29]:
con.close()
print('Daily Lineup Table Updated')

Daily Lineup Table Updated


# Name Mapping

In [33]:
import pandas as pd
import requests
from io import StringIO
import config
import sqlite3

ImportError: No module named config

In [2]:
name_map_url = "http://crunchtimebaseball.com/master.csv"
name_map_content = requests.get(name_map_url).content.decode('utf-8','ignore')
name_map = pd.read_csv(StringIO(name_map_content))

In [ ]:
con = sqlite3.connect(config.player_mapping_db)
existing_players = pd.read_sql('select * from {}'.format(config.player_map_table), con=con)
final_map = existing_players.append(name_map).drop_duplicates()

In [6]:
final_map.to_sql(con=con, name=config.player_map_table, if_exists='replace', index=False)
print("Player ID's updated")
con.close()

In [29]:
lineups.merge(final_map[['rotowire_name','retro_id']], left_on='name',right_on='rotowire_name')

,fixture,name,position,team,game_date,pull_time,rotowire_name,retro_id
0,1,Wei-Yin Chen,P,MIA,2019-03-15,2019-03-15 10:43:45,Wei-Yin Chen,chenw001
1,1,Lewis Brinson,CF,MIA,2019-03-15,2019-03-15 10:43:45,Lewis Brinson,brinl001
2,1,Brian Anderson,3B,MIA,2019-03-15,2019-03-15 10:43:45,Brian Anderson,andeb006
3,1,Martin Prado,1B,MIA,2019-03-15,2019-03-15 10:43:45,Martin Prado,pradm001
4,1,Peter O'Brien,RF,MIA,2019-03-15,2019-03-15 10:43:45,Peter O'Brien,obrip002
5,1,Bryan Holaday,C,MIA,2019-03-15,2019-03-15 10:43:45,Bryan Holaday,holab001
6,1,Yadiel Rivera,SS,MIA,2019-03-15,2019-03-15 10:43:45,Yadiel Rivera,rivey001
7,1,Deven Marrero,2B,MIA,2019-03-15,2019-03-15 10:43:45,Deven Marrero,marrd001
8,1,Sean Newcomb,P,ATL,2019-03-15,2019-03-15 10:43:45,Sean Newcomb,newcs001
9,1,Ozzie Albies,2B,ATL,2019-03-15,2019-03-15 10:43:45,Ozzie Albies,albio001


In [35]:
dc = requests.get('http://oakland.athletics.mlb.com/team/depth_chart/?c_id=oak')

In [36]:
tree = html.fromstring(dc.content)

In [43]:
teams = tree.xpath("//div[@id='depth_chart']")
teams0 = teams[0]
positions = teams0.xpath("div[contains(@id,'pos_')]")
positions0 = positions[0]

In [45]:
positions

[<Element div at 0x11511c100>,
 <Element div at 0x11511c310>,
 <Element div at 0x11511c2b8>,
 <Element div at 0x11511c158>,
 <Element div at 0x11511c470>,
 <Element div at 0x11511c4c8>,
 <Element div at 0x11511c578>,
 <Element div at 0x11511c6d8>,
 <Element div at 0x11511c838>,
 <Element div at 0x11511c890>,
 <Element div at 0x11511c8e8>,
 <Element div at 0x11511c998>,
 <Element div at 0x11511c9f0>]

In [31]:
teams0

'LAA'

In [18]:
positions0.xpath('div/text()')

['Catcher']

In [7]:
for player in positions0.xpath('ol/li/a'):
    print(player.attrib['href'].split('id=')[-1])

9859
13663
13635


In [96]:
depth_charts = []
base_url = 'http://oakland.athletics.mlb.com/team/depth_chart/?c_id={}'
teams = ['oak','ana','hou','tor','atl','mil','stl','chc','ari','la','sf','cle','sea','mia','nym',
        'was','bal','sd','phi','pit','tex','tb','bos','cin','col','kc','det','min','cws','nyy']
for team_name in teams:
    dc = requests.get(base_url.format(team_name))
    tree = html.fromstring(dc.content)
    team_dc = []
    team_page_dc = tree.xpath("//div[@id='depth_chart']")[0]
    positions = team_page_dc.xpath("div[contains(@id,'pos_')]")
    for position in positions:
        try:
            position_name = position.xpath("ul/li[@class='position_header']/text()")[0]
            players = []
            player_els = position.xpath("ul/li/a[@target='_blank']")
            for player in player_els:
                player_name = player.text
                player_id = int(player.attrib['href'].split('id=')[-1])
                player_dict = {'player_name':player_name,
                               'player_id':player_id,
                              'position':position_name,
                              'team_name':team_name.upper()}
                depth_charts.append(player_dict)
        except: # The final one on every page breaks. This acconts for that
            pass

In [97]:
import json

In [ ]:
CREATE TABLE IF NOT EXISTS "depth_charts" (
  "player_id" INTEGER,
  "player_name" TEXT,
  "position" TEXT,
  "team_name" TEXT(3),
  "pull_time" TEXT
);

In [99]:
pd.DataFrame.from_dict(depth_charts)

,player_id,player_name,position,team_name
0,543257,R. Grossman,LEFT FIELD,OAK
1,592192,M. Canha,LEFT FIELD,OAK
2,640461,C. Pinder,LEFT FIELD,OAK
3,657656,R. Laureano,CENTER FIELD,OAK
4,641583,D. Fowler,CENTER FIELD,OAK
5,620439,F. Barreto,CENTER FIELD,OAK
6,592192,M. Canha,CENTER FIELD,OAK
7,572039,S. Piscotty,RIGHT FIELD,OAK
8,640461,C. Pinder,RIGHT FIELD,OAK
9,592192,M. Canha,RIGHT FIELD,OAK


In [74]:
depth_charts

{'OAK': {}}